In [4]:
# run to pretty-print results
import numpy as np
import warnings
np.set_printoptions(precision=5, suppress=True)
warnings.filterwarnings("ignore", category=DeprecationWarning)

Install ``average-minimum-distance`` with pip:
```shell
$ pip install average-minimum-distance
```


``amd.compare()`` compares crystals from CIF files by AMD or PDD descriptors, for example

In [5]:
import amd

# compare items in example_crystals.cif pairwise by AMD, k=100
df = amd.compare('example_crystals.cif', by='AMD', k=100)
print(df, '\n')
# compare item in this cif against items in the other by PDD, k=100
df = amd.compare('example_crystal_1_v2.cif', 'example_crystals.cif', by='PDD', k=100)
print(df)

           crystal_1  crystal_2  crystal_3  crystal_4
crystal_1   0.000000   1.319907   0.975221   2.023880
crystal_2   1.319907   0.000000   0.520115   0.703973
crystal_3   0.975221   0.520115   0.000000   1.072211
crystal_4   2.023880   0.703973   1.072211   0.000000 

              crystal_1  crystal_2  crystal_3  crystal_4
crystal_1_v2    0.05948   1.353296   1.001504   2.046805


The distance matrix returned is a pandas DataFrame. ``amd.compare()`` can also accept paths to folders or lists of paths.

``amd.compare()`` reads crystals from CIFs, calculates their descriptors and compares them, but these steps can be done separately if needed (see below). ``amd.compare()`` accepts several optional parameters, see [the documentation for a full list](https://average-minimum-distance.readthedocs.io/en/latest/Getting_Started.html#list-of-optional-parameters).

# Reading cifs

To read CIF files, use ```amd.CifReader``` (the .cifs used in this notebook can be found in the examples folder). If any structure in a cif cannot be read into a periodic set, it is skipped and a warning is printed. The reader yields ```PeriodicSet``` objects, which can be handed directly to the AMD/PDD calculator functions below.

In [4]:
import amd

# Read a cif with more than one structure into a list
periodic_sets = list(amd.CifReader('example_crystals.cif'))

# Lazily reads a cif with more than one structure
for periodic_set in amd.CifReader('example_crystals.cif'):
    print(periodic_set)

# .read() returns one periodic set if there is exactly one, otherwise returns a list.
structure = amd.CifReader('example_crystals.cif').read()

PeriodicSet(crystal_1: 92 points (92 asym) in 3 dims, abcαβγ=23.22,23.23,7.28,90,90,120)
PeriodicSet(crystal_2: 92 points (92 asym) in 3 dims, abcαβγ=7.25,13.04,20.66,72.46,86.35,74.04)
PeriodicSet(crystal_3: 184 points (184 asym) in 3 dims, abcαβγ=22.32,22.32,7.33,90,90,90)
PeriodicSet(crystal_4: 184 points (184 asym) in 3 dims, abcαβγ=21.49,7.28,14.91,90,81.69,90)


Setting ```remove_hydrogens=True``` will remove Hydrogen atoms. See https://average-minimum-distance.readthedocs.io/en/latest/Reading_cifs.html for a list of all optional arguments, e.g. for handling disorder.

In [5]:
import amd

# T2_gamma.cif contains one crystal
c1_withH = amd.CifReader('example_crystal_1_v2.cif').read()
print(c1_withH.types)  # atomic numbers
c1_noH = amd.CifReader('example_crystal_1_v2.cif', remove_hydrogens=True).read()
print(c1_noH.types)

[8 7 6 6 6 6 6 1 1 1 8 7 6 6 6 6 1 1 8 7 6 6 6 6 1 1 7 6 6 6 6 1 1 1 7 6 6
 6 1 1 7 6 6 6 1 1 8 7 6 6 6 6 6 1 1 1 8 7 6 6 6 6 1 1 8 7 6 6 6 6 1 1 7 6
 6 6 6 1 1 1 7 6 6 6 1 1 7 6 6 6 1 1]
[8 7 6 6 6 6 6 8 7 6 6 6 6 8 7 6 6 6 6 7 6 6 6 6 7 6 6 6 7 6 6 6 8 7 6 6 6
 6 6 8 7 6 6 6 6 8 7 6 6 6 6 7 6 6 6 6 7 6 6 6 7 6 6 6]


# Calculating AMDs and PDDs

The functions ```amd.AMD``` and ```amd.PDD``` take a crystal (and an integer parameter ```k```) and calculate a descriptor. The crystal can be either an output of a ``CifReader``, or you can manually make a pair of NumPy arrays (motif, cell). ``k`` is the number of neighbouring atoms to consider for each atom in a unit cell; generally ``k=100`` is a sensible value to make structurally similar crystals have close distances.

An AMD is a vector length ``k``, while a PDD is a matrix with ``k+1`` columns (the first column contains weights of each row) and generally one row for each symmetrically unique motif point.

In [19]:
import amd

k = 100
# crystal_1_v2.cif contains one crystal
gamma = amd.CifReader('example_crystal_1_v2.cif').read()
gamma_amd = amd.AMD(gamma, k)
print(gamma_amd)

# List of amds from a .cif with several structures
exp_structures = amd.CifReader('example_crystals.cif')
exp_amds = [amd.AMD(periodic_set, k) for periodic_set in exp_structures]

[1.09496 1.6112  1.63668 2.16994 2.32097 2.46125 2.5297  2.68423 2.76638
 2.94424 3.14361 3.34288 3.43191 3.60956 3.71822 3.89075 3.96334 4.07937
 4.13787 4.32556 4.4114  4.49442 4.58891 4.65277 4.69401 4.78252 4.84996
 4.91108 4.96605 5.07076 5.19625 5.29167 5.34259 5.39523 5.48512 5.59889
 5.66037 5.74949 5.81719 5.87116 5.92866 5.96055 6.03031 6.11592 6.15089
 6.21287 6.24386 6.35346 6.3785  6.42105 6.50781 6.54674 6.5997  6.64295
 6.72106 6.81119 6.85539 6.88668 6.92079 6.94661 6.96826 7.02369 7.0654
 7.11497 7.17574 7.21257 7.28026 7.34532 7.40293 7.45709 7.45987 7.51745
 7.56279 7.59403 7.62179 7.68789 7.71134 7.78182 7.80129 7.83915 7.87106
 7.91019 7.94117 7.97491 7.98909 8.03783 8.10649 8.13512 8.15912 8.24342
 8.26936 8.31808 8.34697 8.4137  8.43892 8.47465 8.48789 8.51937 8.58087
 8.66164]


In [18]:
import amd

# The PDD interface is the same as AMD
gamma = amd.CifReader('example_crystal_1_v2.cif').read()
gamma_pdd = amd.PDD(gamma, 100)
print('PDD dims:', gamma_pdd.shape)
print('weights:', gamma_pdd[:, 0])  # first column has weights
print('distances:', gamma_pdd[:, 1:])  # rest of the matrix is distances

PDD dims: (40, 101)
weights: [0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.04348
 0.04348 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174
 0.04348 0.04348 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174
 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.02174 0.04348 0.02174
 0.02174 0.02174 0.02174 0.04348]
distances: [[0.89126 1.36137 1.38153 ... 8.93477 9.01536 9.03127]
 [0.89126 1.92567 1.96379 ... 8.82148 8.82157 8.92923]
 [0.89126 1.36137 1.38153 ... 8.93477 9.01721 9.03127]
 ...
 [1.37527 1.38175 1.39694 ... 8.58104 8.61946 8.61976]
 [1.37527 1.38175 1.39694 ... 8.58104 8.61946 8.61976]
 [1.3753  1.38179 1.39694 ... 8.58106 8.61955 8.61981]]


In [20]:
import amd
import numpy as np

# AMD/PDD accepts a tuple (motif, cell) of arrays of Cartesian coordinates
motif = np.array([[0,0,0]]) # one point at the origin
cell = np.identity(3)  # cubic unit cell
cubic_pdd = amd.PDD((motif, cell), 10)
print(cubic_pdd)

[[1.      1.      1.      1.      1.      1.      1.      1.41421 1.41421
  1.41421 1.41421]]


# Comparing AMDs and PDDs

There are several functions which compare by AMD or PDD. The names 'cdist' and 'pdist' come from SciPy: cdist is for comparing one set against another, whereas pdist takes one set and compares it pairwise.  So to compare with AMDs, use ```amd.AMD_pdist``` or ```amd.AMD_cdist``` and to compare with PDDs use ```amd.PDD_pdist``` or ```amd.PDD_cdist```. The code below compares T2_gamma.cif to structures in T2_experimental.cif by AMD with k = 100:

In [24]:
import amd
import numpy as np

k = 100
crystal_1 = amd.CifReader('example_crystal_1_v2.cif').read()
example_structures = list(amd.CifReader('example_crystals.cif'))
crystal_1_amd = amd.AMD(crystal_1, k)
example_amds = [amd.AMD(s, k) for s in example_structures]
# compare crystal_1 AMD against the others
dm = amd.AMD_cdist(crystal_1_amd, example_amds)
# print structs in T2_experimental in order of distance to T2_gamma 
for i in np.argsort(dm)[0]:
    print(example_structures[i].name, dm[0][i])

crystal_1 0.022073635461426155
crystal_3 0.9904434092760033
crystal_2 1.3351299035517776
crystal_4 2.0391032569149043
crystal_5 2.220926933111067


The output of these functions mimic scipy's [cdist](scipy.spatial.distance.cdist) and [pdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html), cdist returns a 2D distance matrix and pdist returns a 'condensed distance vector' ([see scipy's squareform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.squareform.html)).

These functions take the same optional arguments. If you want to change the defaults to see the effect on comparisons, details are in the readme and documentation. Briefly, ```metric``` (default 'chebyshev', aka l-infinity) is the metric used to compare AMDs or the rows of PDDs, it can be any metric accepted by [scipy's cdist function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html). ```k``` (default None) is the value of k to use when comparing, it just truncates the inputs to a smaller length if an int. 

 The functions that use PDD also take a parameter ```verbose``` which if true will print an ETA to the terminal.

In [25]:
import amd

k = 100
example_structs = list(amd.CifReader('example_crystals.cif'))
example_pdds = [amd.PDD(periodic_set, k) for periodic_set in example_structs]

# compare experimental structures pairwise, return a condensed distance matrix
cdm = amd.PDD_pdist(example_pdds)
print('condensed distance matrix:\n', cdm)

# This converts the condensed distance matrix to a 2D symmetric distance matrix
from scipy.spatial.distance import squareform
dm = squareform(cdm)
print('symmetric 2D distance matrix:\n', dm)

condensed distance matrix:
 [1.34387 0.98718 2.03303 2.21326 0.74984 0.78152 0.93999 1.1476  1.29733
 0.45593]
symmetric 2D distance matrix:
 [[0.      1.34387 0.98718 2.03303 2.21326]
 [1.34387 0.      0.74984 0.78152 0.93999]
 [0.98718 0.74984 0.      1.1476  1.29733]
 [2.03303 0.78152 1.1476  0.      0.45593]
 [2.21326 0.93999 1.29733 0.45593 0.     ]]
